In [1]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

In [2]:
# We have a toy simulation which uses a hand-fixed probability for each game. Over n simulations as n-> inf, the mean
# points for each team should approach the league average. A better simulation will look at two teams playing, determine the 
# probability of each team winning (and the OT win/loss) then feed these unique probabilities into each matchup of the season
# simulator. Currently, there is another piece of code 'Problem3' in AQM/Application which assumes goals are poisson distributed
# It then takes each team as a parameter plus a home/away flag as parameters before minimizing on the 2013/2014 season.

# Here, we will load in the vector of parameters found from minimizing the log-likelihood of this function on the training data
# Then the posson function is used to find probability of each team winning a given matchup.

In [13]:
# Load in some data and define a couple things
df = pd.read_csv("NHL_20132014.csv")           #This has game data and is mostly what is used in the toy simulation
df2 = pd.read_csv("NHL_20132014_processed.csv")

# For each team name give them a W-L-T initial array. For overtime to happen win+loss must sum to less than 1.
# Load in the probability file generated by the Poisson.ipynb regression
#prob = [0.5, 0.4, 0.55]  # Probablility of [Home win, home loss, home OT win] 

prob = np.genfromtxt('game_probs.csv', skip_header=1, delimiter=',')

teams = df2.columns[0:-2] 
for i in range(3):
    print(prob[i])
    print(prob[i, 0], prob[i, 1], prob[i, 2])


# I did a silly billy here where I added the final game of the season to the non-processed datafile but not the processed.
# If I fix that and rerun the regression again, it should be resolved with 1230 games for each file.
len(df)

[0.35 0.33 0.32]
0.35 0.33 0.32
[0.42 0.29 0.29]
0.42 0.29 0.29
[0.34 0.33 0.33]
0.34 0.33 0.33


1230

In [14]:
# This function simulates a single season and returns a [num_teams, 3] array 'score' where 0: wins, 1: losses, 2: OT_losses

def season(schedule, one_hot, prob):
    home, away = schedule['home'], schedule['away']
    teams = one_hot.columns[0:-2]
    games = len(schedule)
    team_no = len(teams)
    score = np.zeros((len(teams), 3)) 
    
    for i in range(0, games):
        roll = random.rand()
        win = prob[i,0]
        loss = prob[i,1]
        win_OT = prob[i,2]*0.55  # For now set home OT win prob fixed at 55%
        
        home_index = one_hot.columns.get_loc(home[i])    # Using get_loc on a dataframe is much faster!
        away_index = one_hot.columns.get_loc(away[i])    # Factor of 6-8 decrease in computation time. 
           
        if roll <= win:
            score[home_index] = score[home_index]+[1,0,0]                  # Home regulation win
            score[away_index] = score[away_index]+[0,1,0]
        elif roll > win and roll <= win+loss:
            score[home_index] = score[home_index]+[0,1,0]                   # Away regulation win
            score[away_index] = score[away_index]+[1,0,0]
        else:
            roll_OT = random.rand()
            if roll_OT <= win_OT:
                score[home_index] = score[home_index]+[1,0,0]                  # Home OT win
                score[away_index] = score[away_index]+[0,0,1]
            else: 
                score[home_index] = score[home_index]+[0,0,1]                   # Away OT win
                score[away_index] = score[away_index]+[1,0,0]
    return score

In [15]:
# This function turns (wins, losses, OT losses) into total points for a team
def sum_points(score):
    pts = np.zeros(len(score))
    j = 0
    for i in score:
        pts[j] = sum(i*[2,0,1])
        j = j+1
    return pts

In [31]:
# This function runs n seasons and returns the mean points and std. for each team over those n seasons

def monte_carlo(n, schedule, one_hot, num_teams, prob):
    result_matrix = np.zeros([n, num_teams]) # Create an n x #teams matrix. Each season results will stored on a single row of this matrix 

    for i in range(0,n):
        score = season(schedule, one_hot, prob)
        result_matrix[i] = sum_points(score)

    mean_points = np.zeros([num_teams, 1])    
    std_points = np.zeros([num_teams, 1])


    for i in range(0, num_teams):
        mean_points[i] = np.mean(result_matrix[:,i]) # Do a transposition to make the array make more sense in my head
        std_points[i] = np.std(result_matrix[:,i])
    return mean_points, std_points

In [45]:
%%time
# Huh. I feel like there was supposed to be a bunch more stuff here but maybe it was deleted? That's unfortunate.
# Luckily, I've set it up so I just need to run monte_carlo here

n = 100
points, error = monte_carlo(n, df, df2, num_teams, prob)

for i in range(len(points)):
    print(teams[i], points[i, 0], error[i, 0])

ANA 103.41 8.002618321524524
BOS 103.31 7.674236118337772
BUF 78.29 7.318872864041292
CAR 93.54 7.529169940969589
CBJ 96.57 8.076205792325997
CGY 91.43 8.015304111510678
CHI 102.15 7.214395331557593
COL 98.65 8.310685892271469
DAL 98.42 8.885021102957493
DET 94.37 6.734471025997514
EDM 91.11 7.915674323770528
FLA 89.12 8.563036844484555
LAK 91.62 8.495622402155124
MIN 92.61 8.691254224794026
MTL 93.1 8.510581648747635
NJD 89.83 7.409527650262196
NSH 94.53 7.139264668017288
NYI 93.53 8.713730544376501
NYR 95.69 7.431951291551902
OTT 96.45 8.84350043817492
PHI 99.82 8.446750854618598
PHX 92.36 8.485894177987374
PIT 99.51 7.771093874095203
SJS 100.14 7.717538467672189
STL 99.51 8.369581829458388
TBL 96.34 8.248902957363482
TOR 96.26 8.621623976954691
VAN 87.14 8.874705628920882
WPG 94.02 9.008862303310003
WSH 95.62 7.900354422429414
Wall time: 3.5 s


In [41]:


m = np.mean(a)
s = np.std(a)
print(m, s)

102.94333333333333 0.7931932649459114
